<a href="https://colab.research.google.com/github/afrahhassan/afrahhassan/blob/main/assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Reading Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

# share link or share drive
!wget -O file_name.txt "https://drive.google.com/file/d/1UttAo3Ml9KyWkMybABFRy_im0GdTgi7Q/view?usp=sharing"

--2024-04-28 05:15:02--  https://drive.google.com/file/d/1UttAo3Ml9KyWkMybABFRy_im0GdTgi7Q/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 142.251.8.100, 142.251.8.139, 142.251.8.113, ...
Connecting to drive.google.com (drive.google.com)|142.251.8.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘file_name.txt’

file_name.txt           [ <=>                ]  83.64K  --.-KB/s    in 0.03s   

2024-04-28 05:15:03 (2.98 MB/s) - ‘file_name.txt’ saved [85647]



In [ ]:
!cp "/content/drive/MyDrive/ML/Assignment2" .

cp: -r not specified; omitting directory '/content/drive/MyDrive/ML/Assignment2'


In [ ]:
!cp -r "/content/drive/MyDrive/ML/Assignment2" .

cp: cannot open '/content/drive/MyDrive/ML/Assignment2/images.gsheet' for reading: Operation not supported


In [ ]:
import zipfile
zip_file_path = '/content/drive/My Drive/ML/Assignment2/trafficsigns_dataset.zip'
extract_to_path = '/content/trafficsigns_dataset'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)
print("Extraction completed!")

Extraction completed!


In [ ]:
import os
import csv

root_dir = '/content/trafficsigns_dataset'

output_csv = '/content/drive/My Drive/ML/Assignment2/images.csv'

with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['image_path', 'class'])

    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                full_path = os.path.join(root, file)
                class_name = os.path.basename(root)
                writer.writerow([full_path, class_name])
print("CSV file has been created successfully.")

CSV file has been created successfully.


In [ ]:
# Set the path for the output CSV file
output_csv = '/content/drive/My Drive/ML/Assignment2/image2.csv'  # Corrected the filename extension from '.cs' to '.csv'
root_dir = '/content/trafficsigns_dataset'  # Directory containing subfolders of images

# This list will hold all unique folder names which correspond to shape classes
class_names = []

# Opening a file to write into, make sure you have the correct permissions or the path exists
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['image_path', 'shape'])  # Define the column headers for image paths and shape types

    # Walk through the directory structure
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            # Check for image files ending with .png, .jpg, or .jpeg
            if file.endswith(('.png', '.jpg', '.jpeg')):
                full_path = os.path.join(root, file)  # Full path to the image
                shape_name = os.path.basename(root)  # Folder name is taken as the shape name

                # If this shape_name is not already in the list, add it
                if shape_name not in class_names:
                    class_names.append(shape_name)

                # Write the path and shape_name to the CSV file
                writer.writerow([full_path, shape_name])

print("CSV file has been created successfully.")
print("Detected shape classes:", class_names)


CSV file has been created successfully.
Detected shape classes: ['laneend', 'continue', 'crossing', 'parking', 'trafficdirective', 'limitedtraffic', 'bicycle', 'roundabout', 'noentry', 'speed', 'traveldirection', 'noparking', 'giveway', 'warning', 'rightofway', 'stop']
